<a href="https://colab.research.google.com/github/wisrovi/RedesNeuronales/blob/master/AprendizajePorRefuerzo-PongDeterministic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoría

## Objetivo: 
Repasar los conceptos vistos en clase.

La puntuación de este bloque es de 4 puntos sobre la nota final.

**Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?**

**Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.**

**Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.**




**Lista tres diferencias entre los algoritmos de DQN y Policy Gradient**

# Práctica

Objetivo: Implementar una solución, usando keras-rl y basada en el algoritmo de DQN visto en clase, para que un agente aprenda una estrategia ganadora en el juego del Pong.

La puntuación de este bloque es de 6 puntos sobre la nota final.

El entorno sobre el que trabajaremos será _PongDeterministic-v0_ y el algoritmo que usaremos será _DQN_.

Para evaluar cómo lo está haciendo el agente, la recompensa en el _Pong_ oscila, aproximadamente, en el rango de valores **[-20, 20]**. La estrategia óptima de un agente estaría alrededor de una media de recompensa de 20.

- **NOTA IMPORTANTE**: Si el agente no llegara a aprender una estrategia ganadora, responder sobre la mejor estrategia obtenida.

# Drive

In [0]:
isColabGoogle = False

In [0]:
if isColabGoogle:
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_FOLDER = '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/'
else:
    BASE_FOLDER = ""

# Install and import

In [3]:
!pip install keras-rl==0.4.2
!pip install tensorflow==1.13.1
!pip install gym
!pip install gym[atari]
!pip install h5py


#creo son necesarias, pero no se ha comprobado que en su ausencia falle
!pip install jupyter
!pip install torch

In [4]:
from PIL import Image

import numpy as np
import gym

# Para las librerias para la red neuronal
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam, SGD

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [5]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# random seed
np.random.seed(123)
env.seed(123)

[123, 151010689]

In [0]:
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [0]:
processor = PongProcessor()

# CNN

In [0]:
def createModel(transferModel = None):
    input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

    model = Sequential()
    if transferModel is not None:
        model.add(transferModel)
    else:
        model.add(Permute((2, 3, 1), input_shape=input_shape))
    
    #model.add( Convolution2D(8, kernel_size=(16, 16), strides=(5, 5), activation='relu', padding='same'  ) )
    model.add( Convolution2D(16, kernel_size=(16, 16), strides=(5, 5), activation='relu', padding='same'  ) )
    model.add( Convolution2D(32, kernel_size=(8, 8), strides=(4, 4), activation='relu', padding='same'  ) )
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add( Convolution2D(64, kernel_size=(4, 4), strides=(2, 2), activation='relu', padding='same'  ) )
    model.add( Convolution2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'  ) )    
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.25)) #apagar un 25% de manera aleatoria para reducir la cantidad de parametros
    model.add(Flatten())
    model.add(Dense(nb_actions, activation='linear'))   #model.add(Dense(nb_actions, activation='sigmoid'))    
    return model

In [9]:
model = createModel()
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 16)        16400     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 32)          32800     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          32832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         73856     
________________________________________________

# Memory and policy

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

# Agente

## Create

In [0]:
dqn = DQNAgent(
    nb_actions=nb_actions, 
    model=model, 
    policy=policy, 
    memory=memory,
    processor=processor, 
    nb_steps_warmup=50000, 
    target_model_update=10000,
    gamma=.99)

## Optimizer

In [0]:
optimizer = Adam(lr=1e-3)
optimizer = SGD()

## Compile

In [0]:
#compilar modelo
dqn.compile(optimizer, metrics=['mae'])

## callbacks

In [0]:
callbacks = list()

In [0]:
#Para algunos casos es importante saber cual entrenamiento fue mejor, 
#este callback guarda el modelo tras cada epoca completada con el fin de si luego se desea un registro de pesos para cada epoca
#Se ha usado este callback para poder optener el mejor modelo de pesos, sobretodo en la red neuronal creada desde cero
#siendo de gran utilidad para determinar el como ir modificando los layer hasta obtener el mejor modelo
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
guardarModeloCadaCiertoIntervaloDeSteps = ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)

callbacks.append(guardarModeloCadaCiertoIntervaloDeSteps)

In [0]:
log_filename = 'dqn_{}_log.json'.format(ENV_NAME)
guardarModelo = FileLogger(log_filename, interval=100)

callbacks.append(guardarModelo)

In [0]:
#EarlyStopping, detener el entrenamiento una vez que su pérdida comienza a aumentar 
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=5, #argumento de patience representa el número de épocas antes de detenerse una vez que su pérdida comienza a aumentar (deja de mejorar). 
    min_delta=0,  #es un umbral para cuantificar una pérdida en alguna época como mejora o no. Si la diferencia de pérdida es inferior a min_delta , se cuantifica como no mejora. Es mejor dejarlo como 0 ya que estamos interesados ​​en cuando la pérdida empeora.
    mode='auto')

callbacks.append(early_stop)

In [0]:
#ReduceLROnPlateau, que si el entrenamiento no mejora tras unos epochs específicos, reduce el valor de learning rate del modelo
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=5, 
    epsilon=1e-4, 
    mode='min')

callbacks.append(reduce_lr)

In [50]:
len(callbacks)

2

# Fit

In [0]:
entrenar = True
usarPesosEntrenados = True
testearAlFinalEntrenamiento = False  #si se hace un testeo, se guarda solo si hay mejoria

In [0]:
totalSteps = 200000
intervalo = 10000

cantidadEntrenamientos = 10 #cantidad de veces que repito el entrenamiento con el totalSteps

#si totalSteps = 100000 y intervalo = 10000, entonces se haran 10 intervalos

In [0]:
#archivo guardar pesos
weights_filename = BASE_FOLDER + 'dqn_' + ENV_NAME + '_weights2.h5f'

In [0]:
if usarPesosEntrenados:
    try:
        dqn.load_weights(BASE_FOLDER + weights_filename)
        scoreOld = dqn.test(env, nb_episodes=2, visualize=False).history['episode_reward'][1]
        print(dqn.test(env, nb_episodes=2, visualize=False).history['episode_reward'][1])
    except:
        scoreOld = -50
        print("No se encontro archivos de pesos")
        pass

if entrenar:
    agenteOld = None
    for i in range(cantidadEntrenamientos):
    # Training part
        try:
            dqn.fit(env, callbacks=callbacks, nb_steps=totalSteps, log_interval=intervalo, visualize=False)
        except:
            print("Error al entrenar, se ha interrumpido el fit")
            pass
       
        if testearAlFinalEntrenamiento:
            testEval = dqn.test(env, nb_episodes=2, visualize=False).history['episode_reward'][1]       

            if scoreOld < testEval:
                scoreOld = testEval
                agenteOld = dqn
                dqn.save_weights(BASE_FOLDER + weights_filename, overwrite=True)
                print("Guardando pesos", scoreOld)
            else:
                print("No hubo mejora, no se guarda este modelo.")
                conteo += 1
                if conteo > 3:
                    pass
                    break
                dqn = agenteOld   
        else:
            dqn.save_weights(BASE_FOLDER + weights_filename, overwrite=True)
else:    
    dqn.load_weights(weights_filename)

No se encontro archivos de pesos
Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.0213
10 episodes - episode_reward: -20.000 [-21.000, -19.000] - ale.lives: 0.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.0202
10 episodes - episode_reward: -20.100 [-21.000, -18.000] - ale.lives: 0.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.0212
11 episodes - episode_reward: -20.273 [-21.000, -19.000] - ale.lives: 0.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -0.0201
10 episodes - episode_reward: -19.900 [-21.000, -18.000] - ale.lives: 0.000

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 50s 5ms/step - reward: -0.0216
10 episodes - episode_reward: -20.400 [-21.000, -18.000] - ale.lives

## Test

In [0]:
eposidiosEvaluar = 10

In [0]:
rta = dqn.test(env, nb_episodes=eposidiosEvaluar, visualize=False).history['episode_reward'][1]

#print(rta)